In [1]:
import pandas as pd
#import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
#import matplotlib.animation as animation
import numpy as np
import os
import seaborn
import scipy.stats as stats
from sklearn.neighbors import KernelDensity

In [2]:
def getthemean(filename,colver='iteration',whichcol=['hr_top10']):
    data=pd.read_csv(filename)
    num=int(np.max(data[colver]))
    meandata=data[data[colver]==1][whichcol].get_values()
    for i in range(2,num+1):
        meandata+=data[data[colver]==i][whichcol].get_values()
    meandata/=num
    return meandata

In [3]:
def getone(filename,colver='iteration',whichcol=['hr_top10'],iterval=1):
    data=pd.read_csv(filename)
    return data[data[colver]==iterval][whichcol].get_values()

In [4]:
def getfiles(start="bhist",items='120',mis=0):
    listoffiles=[]
    where='../DataDump'
    for file in os.listdir(where):
        if file.startswith(start):
            if items in file:
                if mis==0:
                    if '50' not in file:
                        listoffiles.append(file)
                elif mis==1:
                    if '50' in file:
                        listoffiles.append(file)
                else:
                    listoffiles.append(file)
    return where,listoffiles

In [5]:
def getmaxmin(filename,colver='iteration',whichcol=['hr_top10']):
    data=pd.read_csv(filename)
    num=int(np.max(data[colver]))
    maxmindata=data[data[colver]==1][whichcol].get_values()
    if maxmindata.ndim==2:
        n,m=np.shape(maxmindata)
        maxmindata=np.empty((n,m,num))
        for i in range(1,num+1):
            maxmindata[:,:,i-1]=data[data[colver]==i][whichcol].get_values()
        sortedmaxmin=np.sort(maxmindata,-1)
        return sortedmaxmin[:,:,4],np.mean(sortedmaxmin,-1),sortedmaxmin[:,:,-5]
    else:
        n=np.shape(maxmindata)[0]
        maxmindata=np.empty((n,num))
        for i in range(1,num+1):
            maxmindata[:,i-1]=data[data[colver]==i][whichcol].get_values()
        sortedmaxmin=np.sort(maxmindata,-1)
        return sortedmaxmin[:,4],np.mean(sortedmaxmin,-1),sortedmaxmin[:,-5]

In [6]:
def createlabel(f):
    name=f.split()
    label=[]
    typeoflab=name[1][1:]
    if typeoflab=='TS' or typeoflab=='greedy':
        label.append(typeoflab)
        label.append(' $\epsilon$=')
        label.append(str(int(name[14])/int(name[6]))[:5])
        label.append(' $\delta$=')
        label.append(name[15][3:])
    elif typeoflab=='learnthenearn':
        label.append(typeoflab)
        label.append(' $\\alpha$=')
        label.append(name[-2])
        #if name[-5]!='0':
        #    label.append(' misinf')
    elif typeoflab=='fixed_sparse':
        label.append('Fixed Express')
        #if name[-2]!='0':
            #label.append(' misinf')
    return ''.join(label)

In [7]:
def maporder(A):
    trueutil=pd.read_csv('../MABMaxDiff/HB_120items.csv').get_values().mean(0)[2:]
    order=list(np.argsort(trueutil)[::-1])
    newfun=np.vectorize(order.index)
    return newfun(A-1)

In [8]:
def histplots(indices=np.arange(0,6),whichcol='hr_top3',items='120',mis=0,name='name.pdf'):
    where,listoffiles=getfiles(start="hrhist",items=items,mis=mis)
    for i,f in enumerate(listoffiles):
        if i in indices:
            meandata=getthemean(where+'/'+f,colver='iteration',whichcol=whichcol)
            label=createlabel(f)
            plt.plot(range(20,1040,20),meandata,label=label)
    plt.ylabel(whichcol)
    plt.xlabel('respondent')
    plt.legend(loc='lower right',fontsize='small')
    #plt.title('Top '+whichcol[6:]+' Hit Rate Over Respondents')
    plt.xlim([0,1050])
    plt.savefig(name, dpi=100)
    plt.close()



In [9]:
def utilplots(indices=np.arange(0,6),whichcol='util3',items='120',mis=0,name='name.pdf'):
    where,listoffiles=getfiles(start="util",items=items,mis=mis)
    for i,f in enumerate(listoffiles):
        if i in indices:
            meandata=getthemean(where+'/'+f,colver='iteration',whichcol=whichcol)
            label=createlabel(f)
            plt.plot(range(20,1040,20),meandata,label=label)
    plt.ylabel(whichcol)
    plt.xlabel('respondent')
    plt.legend(loc='lower right',fontsize='small')
    #plt.title('Utility Score For Top '+whichcol[4:]+' Items Over Respondents')
    plt.xlim([0,1050])
    plt.savefig(name, dpi=100)
    plt.close()

In [10]:
def vrplots(indices=np.arange(0,6),whichcol='valrem3',items='120',mis=0,name='name.pdf'):
    where,listoffiles=getfiles(start="valrem",items=items,mis=mis)
    for i,f in enumerate(listoffiles):
        if i in indices:
            meandata=getthemean(where+'/'+f,colver='iteration',whichcol=whichcol)
            label=createlabel(f)
            plt.semilogy(range(20,1040,20),meandata,label=label)
            #plt.plot(range(20,1040,20),meandata,label=label)
            plt.axhline(.02,0, 1,linestyle='dashed')
    plt.ylabel('Potential Value Remaining')
    plt.xlabel('respondent')
    plt.legend(loc='upper right',fontsize='small')
    #plt.title('Potential Value Remaining For Top '+whichcol[6:]+' Items Over Respondents')
    plt.xlim([0,1050])
    plt.savefig(name, dpi=100)
    plt.close()

In [11]:
def gettherank(matrix):
    n,m=np.shape(matrix)
    ranking=np.zeros((n,m))
    for i in range(n):
        array=matrix[i]
        temp = array.argsort()[::-1]
        ranks = np.empty(len(array), int)
        ranks[temp] = np.arange(len(array))+1
        ranking[i]=ranks
    return ranking

In [30]:
def stplots(indices=np.arange(0,6),rank=[1,60],items='120',mis=0,name='name.pdf',sq=1):
    where,listoffiles=getfiles(start="sehist",items=items,mis=mis)
    whichcol=whichcols(rank)
    for i,f in enumerate(listoffiles):
        if i in indices:
            for j,col in enumerate(whichcol):
                meandata=getthemean(where+'/'+f,colver='V1',whichcol=col)
                if sq==0:
                    meandata=1/meandata**2 
                label=createlabel(f)
                plt.plot(range(20,1040,20),meandata,label=label+', '+str(rank[j])+' rank')
    if sq==0:
        plt.ylabel('Precision')
        plt.legend(loc='upper left',fontsize='small')
    else:
        plt.ylabel('Squared Error')
        plt.legend(loc='upper right',fontsize='small')
    plt.xlabel('respondent')
    
    #if sq==0:
        #plt.title('Precision Over Respondents')
    #else:
    #    plt.title('Squared Error Over Respondents')
    plt.xlim([0,1050])
    plt.savefig(name, dpi=100)
    plt.close()

In [13]:
def rundots(index,where,listoffiles):
    data=pd.read_csv(where+'/'+listoffiles[index])
    choices=data.get_values()[:,1:]
    res=maporder(choices)
    return res,createlabel(listoffiles[index][55:])

In [14]:
def trueOrder():
    trueutil=pd.read_csv('../MABMaxDiff/HB_120items.csv').get_values().mean(0)[2:]
    order=list(np.argsort(trueutil)[::-1])
    return order

In [59]:
def whichcols(rank):
    order=trueOrder()
    cols=[]
    for i in rank:
        cols.append('V'+str(order[i-1]+3))
    return cols

In [61]:
def barplots(indices=[0],size=(10,4),name='name.pdf'):
    listoffiles=[]
    where='../DataDump'
    for file in os.listdir(where):
        if file.startswith("itembyresp"):
                    listoffiles.append(file)
    n=len(indices)
    for i,index in enumerate(indices):
        res,label=rundots(index,where,listoffiles)
        plt.subplot('1'+str(n)+str(i+1))
        plt.plot(res,'k,', markersize=1)
        plt.xlim([0,1020])
        plt.title(label)
    fig = plt.gcf()
    fig.set_size_inches(size[0], size[1])
    fig.savefig(name, dpi=100)
    plt.close()

In [62]:
def rankplot(indices=np.arange(0,6),rank=[1,2,3],iterval=1,items='120',mis=0,name='name.pdf',size=(10,6)):
    where,listoffiles=getfiles(start="bhist",items=items,mis=mis)
    whichcol=[]
    for i in range(120):
        whichcol.append('V'+str(i+3))
    n=len(indices)
    order=trueOrder()
    for i,f in enumerate(listoffiles):
        if i in indices:
            num=indices.index(i)+1
            meandata=getone(where+'/'+f,colver='V1',whichcol=whichcol,iterval=iterval)
            therank=gettherank(meandata)
            plt.subplot('1'+str(n)+str(num))
            l,k=np.shape(therank)
            comeback=[]
            for j in range(k):
                if order.index(j)+1 in rank:
                    comeback.append(j)
                else:
                    plt.plot(range(20,1040,20),therank[:,j],color = '0.75')
            for j in comeback:     
                plt.plot(range(20,1040,20),therank[:,j])
            plt.ylim(121,0)
            plt.xlim([20,1020])
            plt.title(createlabel(f))
            if num==1:
                plt.ylabel('rank')
            plt.xlabel('respondent')
    fig = plt.gcf()
    fig.set_size_inches(size[0], size[1])
    fig.savefig(name, dpi=100)
    plt.close()

In [36]:
def bhplots(indices=np.arange(0,6),rank=[1,60],items='120',mis=0,name='name.pdf'):
    where,listoffiles=getfiles(start="bhist",items=items,mis=mis)
    whichcol=whichcols(rank)
    for i,f in enumerate(listoffiles):
        if i in indices:
            for j,col in enumerate(whichcol):
                mindata,meandata,maxdata=getmaxmin(where+'/'+f,colver='V1',whichcol=col)
                label=createlabel(f)
                p=plt.plot(range(20,1040,20),meandata,label=label+', '+str(rank[j])+' rank')
                c=plt.getp(p[0],'color')
                plt.fill_between(range(20,1040,20), mindata, maxdata,facecolor=c, alpha=0.5)
    plt.ylabel('Utility')
    plt.legend(loc='upper right',fontsize='small')
    plt.xlabel('Respondent')
    #plt.title('Beta Values: Mean and 90% Interval')
    plt.xlim([20,1020])
    plt.savefig(name, dpi=100)
    plt.close()

In [19]:
where,filenames=getfiles(start="util",items='120',mis=0)

In [20]:
filenames

['util (fixed_sparse - 120 items - 30 items per resp - misinf 0 ).csv',
 'util (greedy - 120 items - 30 items per resp - split 20 TS 10 TSx10000 - misinf 0 ).csv',
 'util (TS - 120 items - 30 items per resp - split 0 TS 30 TSx1e-04 - misinf 0 ).csv',
 'util (TS - 120 items - 30 items per resp - split 15 TS 15 TSx10 - misinf 0 ).csv',
 'util (TS - 120 items - 30 items per resp - split 15 TS 15 TSx20 - misinf 0 ).csv',
 'util (TS - 120 items - 30 items per resp - split 20 TS 10 TSx10 - misinf 0 ).csv',
 'util (TS - 120 items - 30 items per resp - split 20 TS 10 TSx10000 - misinf 0 ).csv',
 'util (TS - 120 items - 30 items per resp - split 20 TS 10 TSx20 - misinf 0 ).csv',
 'util (TS - 120 items - 30 items per resp - split 30 TS 0 TSx1 - misinf 0 ).csv']

In [51]:
histplots(indices=[0,5,8],whichcol='hr_top3',items='120',mis=0,name='plots/3hitrate120show3.pdf')
histplots(indices=[0,5,8],whichcol='hr_top10',items='120',mis=0,name='plots/10hitrate120show3.pdf')

In [72]:
histplots(indices=[0,5,8],whichcol='hr_top3',items='120',mis=1,name='plots/3hitrate120show3mis.pdf')
histplots(indices=[0,5,8],whichcol='hr_top10',items='120',mis=1,name='plots/10hitrate120show3mis.pdf')


In [47]:
goodones=[0,1,2,5,6,8]

histplots(indices=goodones,whichcol='hr_top3',items='120',mis=0,name='plots/3hitrate120all.pdf')
histplots(indices=goodones,whichcol='hr_top10',items='120',mis=0,name='plots/10hitrate120all.pdf')
histplots(indices=goodones,whichcol='hr_top3',items='120',mis=1,name='plots/3hitrate120allmis.pdf')
histplots(indices=goodones,whichcol='hr_top10',items='120',mis=1,name='plots/10hitrate120allmis.pdf')

histplots(whichcol='hr_top3',items='40',mis=0,name='plots/3hitrate40all.pdf')
histplots(whichcol='hr_top10',items='40',mis=0,name='plots/10hitrate40all.pdf')
histplots(whichcol='hr_top3',items='40',mis=1,name='plots/3hitrate40allmis.pdf')
histplots(whichcol='hr_top10',items='40',mis=1,name='plots/10hitrate40allmis.pdf')

histplots(whichcol='hr_top3',items='300',mis=0,name='plots/3hitrate300all.pdf')
histplots(whichcol='hr_top10',items='300',mis=0,name='plots/10hitrate300all.pdf')
histplots(whichcol='hr_top3',items='300',mis=1,name='plots/3hitrate300allmis.pdf')
histplots(whichcol='hr_top10',items='300',mis=1,name='plots/10hitrate300allmis.pdf')

In [20]:
utilplots(indices=[0,5,8],whichcol='util3',items='120',mis=0,name='plots/3util120show3.pdf')
utilplots(indices=[0,5,8],whichcol='util10',items='120',mis=0,name='plots/10util120show3.pdf')
utilplots(indices=[0,5,8],whichcol='util3',items='120',mis=1,name='plots/3util120show3mis.pdf')
utilplots(indices=[0,5,8],whichcol='util10',items='120',mis=1,name='plots/10util120show3mis.pdf')

In [35]:
goodones=[0,1,2,5,6,8]
utilplots(indices=[0,8],whichcol='util3',items='120',mis=0,name='plots/3util120show2.pdf')
utilplots(indices=[0,8],whichcol='util10',items='120',mis=0,name='plots/10util120show2.pdf')
utilplots(indices=goodones,whichcol='util3',items='120',mis=0,name='plots/3util120all.pdf')
utilplots(indices=goodones,whichcol='util10',items='120',mis=0,name='plots/10util120all.pdf')
utilplots(indices=goodones,whichcol='util3',items='120',mis=1,name='plots/3util120allmis.pdf')
utilplots(indices=goodones,whichcol='util10',items='120',mis=1,name='plots/10util120allmis.pdf')

utilplots(whichcol='util3',items='40',mis=0,name='plots/3util40all.pdf')
utilplots(whichcol='util10',items='40',mis=0,name='plots/10util40all.pdf')
utilplots(whichcol='util3',items='40',mis=1,name='plots/3util40allmis.pdf')
utilplots(whichcol='util10',items='40',mis=1,name='plots/10util40allmis.pdf')

utilplots(whichcol='util3',items='300',mis=0,name='plots/3util300all.pdf')
utilplots(whichcol='util10',items='300',mis=0,name='plots/10util300all.pdf')
utilplots(whichcol='util3',items='300',mis=1,name='plots/3util300allmis.pdf')
utilplots(whichcol='util10',items='300',mis=1,name='plots/10util300allmis.pdf')

In [24]:
vrplots(indices=[0,5,8],whichcol='valrem3',items='120',mis=0,name='plots/3vr120show3.pdf')
vrplots(indices=[0,5,8],whichcol='valrem10',items='120',mis=0,name='plots/10vr120show3.pdf')

In [22]:
goodones=[0,1,2,5,6,8]
vrplots(indices=[0,8],whichcol='valrem3',items='120',mis=0,name='plots/3vr120show2.pdf')
vrplots(indices=[0,8],whichcol='valrem10',items='120',mis=0,name='plots/10vr120show2.pdf')
vrplots(indices=goodones,whichcol='valrem3',items='120',mis=0,name='plots/3vr120all.pdf')
vrplots(indices=goodones,whichcol='valrem10',items='120',mis=0,name='plots/10vr120all.pdf')
vrplots(indices=goodones,whichcol='valrem3',items='120',mis=1,name='plots/3vr120allmis.pdf')
vrplots(indices=goodones,whichcol='valrem10',items='120',mis=1,name='plots/10vr120allmis.pdf')

vrplots(whichcol='valrem3',items='40',mis=0,name='plots/3vr40all.pdf')
vrplots(whichcol='valrem10',items='40',mis=0,name='plots/10vr40all.pdf')
vrplots(whichcol='valrem3',items='40',mis=1,name='plots/3vr40allmis.pdf')
vrplots(whichcol='valrem10',items='40',mis=1,name='plots/10vr40allmis.pdf')

vrplots(whichcol='valrem3',items='300',mis=0,name='plots/3vr300all.pdf')
vrplots(whichcol='valrem10',items='300',mis=0,name='plots/10vr300all.pdf')
vrplots(whichcol='valrem3',items='300',mis=1,name='plots/3vr300allmis.pdf')
vrplots(whichcol='valrem10',items='300',mis=1,name='plots/10vr300allmis.pdf')

In [69]:
stplots(indices=[0,8],rank=[1,60],items='120',mis=0,name='plots/squarederror.pdf',sq=1)
stplots(indices=[0,8],rank=[1,60],items='120',mis=0,name='plots/precision.pdf',sq=0)

In [26]:
barplots([0,3],size=(10,4),name='plots/2dotplot.pdf')
barplots([0,3,1],size=(10,4),name='plots/3dotplot.pdf')

In [63]:
bhplots(indices=[0,8],rank=[1,60],items='120',mis=0,name='plots/betavalconfidence.pdf')

In [89]:
rankplot(indices=[0,5],rank=np.arange(1,11),iterval=9,items='120',mis=0,name='plots/rankingcomp2.pdf')
rankplot(indices=[0,5,3],rank=np.arange(1,11),iterval=9,items='120',mis=0,name='plots/rankingcomp3.pdf')
rankplot(indices=[0,5,3],rank=np.arange(1,11),iterval=9,items='120',mis=1,name='plots/rankingcomp3mis.pdf')

In [32]:
def norplots(indices=np.arange(0,6),rank=[1,60],items='120',mis=0,name='name.pdf',iterval=1,ts=0,after=0,xlim=[-2,5],ylim=[0,8]):
    colors=['b','r','c','g']
    where,listoffiles1=getfiles(start="sehist",items=items,mis=mis)
    where,listoffiles2=getfiles(start="bhist",items=items,mis=mis)
    whichcol=whichcols(rank)
    n=len(rank)
    z=0
    for i,f in enumerate(listoffiles1):
        if i in indices:
            for j,col in enumerate(whichcol):
                sehist=getone(where+'/'+listoffiles1[i],colver='V1',whichcol=col,iterval=iterval)
                bhist=getone(where+'/'+listoffiles2[i],colver='V1',whichcol=col,iterval=iterval)
                x=np.linspace(-3,5,1000)
                label=createlabel(f)
                
                if after:
                    lab='N($\\beta,\sigma$)'
                    plt.fill_between(x,0, stats.norm.pdf(x,bhist[ts], sehist[ts]),
                                 label=lab,
                                 alpha=.9-.5*j,facecolor=colors[(z)%4])
                    lab='N($\\beta, \delta \sigma$), $\delta$=10'
                    plt.fill_between(x,0, stats.norm.pdf(x,bhist[ts], sehist[ts]*10),
                                 label=lab,
                                 alpha=.4,facecolor=colors[(z)%4])
                else:
                    plt.fill_between(x,0, stats.norm.pdf(x,bhist[ts], sehist[ts]),
                                 label=label+', '+str(rank[j])+' rank',
                                 alpha=.9-.5*j,facecolor=colors[(z)%4]) 
            z+=1
    if not after:
        #plt.title('Distribution of items at the ' +str((ts+1)*20)+'th respondent')
        plt.xlabel('Utility')
    else:
        plt.title('Distribution of rank '+str(rank[0])+' item at the ' +str((ts+1)*20)+'th respondent')
    plt.legend(loc='upper right',fontsize='small')
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.savefig(name, dpi=100)
    plt.close()

In [64]:
norplots([0,8],rank=[1,60],ts=5,iterval=5,name='plots/beliefdistrBegin.pdf')
norplots([0,8],rank=[1,60],ts=50,iterval=5,name='plots/beliefdistrEnd.pdf')

In [31]:
norplots([3],rank=[5],ts=5,iterval=5,name='plots/espdeltaBegin.pdf',after=1,xlim=[-2,4],ylim=[0,8])
norplots([3],rank=[5],ts=50,iterval=5,name='plots/espdeltaEnd.pdf',after=1,xlim=[-2,4],ylim=[0,8])

In [33]:
def noranimation(indices=np.arange(0,6),rank=[1,60],items='120',mis=0,name='name',iterval=1,after=0,xlim=[-2,5],ylim=[0,8]):
    colors=['b','r','c','g']
    where,listoffiles1=getfiles(start="sehist",items=items,mis=mis)
    where,listoffiles2=getfiles(start="bhist",items=items,mis=mis)
    whichcol=whichcols(rank)
    n=len(rank)
    m=len(indices)
    sehist=np.zeros((51,m,n))
    bhist=np.zeros((51,m,n))
    label=[]
    z=0
    for i,f in enumerate(listoffiles1):
        if i in indices:
            for j,col in enumerate(whichcol):
                sehist[:,z,j]=getone(where+'/'+listoffiles1[i],colver='V1',whichcol=col,iterval=iterval)
                bhist[:,z,j]=getone(where+'/'+listoffiles2[i],colver='V1',whichcol=col,iterval=iterval)
                
                label.append(createlabel(f))
            z+=1
    fig = plt.figure()
    def updatefig(ts):
        fig.clear()
        x=np.linspace(-3,5,1000)
        for i in range(m):
            for j in range(n):
                if after:
                    lab='N($\\beta,\sigma$)'
                    plt.fill_between(x,0, stats.norm.pdf(x,bhist[ts,i,j], sehist[ts,i,j]),
                                 label=lab,
                                 alpha=.9-.5*j,facecolor=colors[(i)%4])
                    lab='N($\\beta, \delta \sigma$), $\delta$=10'
                    plt.fill_between(x,0, stats.norm.pdf(x,bhist[ts,i,j], sehist[ts,i,j]*10),
                                 label=lab,
                                 alpha=.4,facecolor=colors[(i)%4])
                else:
                    plt.fill_between(x,0, stats.norm.pdf(x,bhist[ts,i,j], sehist[ts,i,j]),
                                             label=label[(i*n+j)]+', '+str(rank[j])+' rank',
                                             alpha=.9-.5*j,facecolor=colors[(i)%4])
        if not after:
            plt.title('Distribution of items at the ' +str((ts+1)*20)+'th respondent')
        else:
            plt.title('Distribution of rank '+str(rank[0])+' item at the ' +str((ts+1)*20)+'th respondent')
        plt.legend(loc='upper right',fontsize='small')
        plt.xlim(xlim)
        plt.ylim(ylim)
        plt.draw()
    
    anim = animation.FuncAnimation(fig, updatefig, 51)
    anim.save(name+'.mp4', fps=10)

In [27]:
noranimation([0,5],rank=[1,90],iterval=5,name='beliefdistr')

In [34]:
noranimation([3],rank=[5],iterval=5,after=1,xlim=[-2,4],ylim=[0,8],name='espdelta')

In [21]:
def rankanim(indices=np.arange(0,6),rank=[1,2,3],iterval=1,items='120',mis=0,name='name.pdf',size=(10,6)):
    where,listoffiles=getfiles(start="bhist",items=items,mis=mis)
    whichcol=[]
    for i in range(120):
        whichcol.append('V'+str(i+3))
    n=len(indices)
    order=trueOrder()
    therank=np.zeros((51,120,n))
    label=[]
    for i,f in enumerate(listoffiles):
        if i in indices:
            num=indices.index(i)+1
            meandata=getone(where+'/'+f,colver='V1',whichcol=whichcol,iterval=iterval)
            therank[:,:,num-1]=gettherank(meandata)
            label.append(createlabel(f))
    fig = plt.figure()
    def updatefig(ts):
        fig.clear()
        for num in range(n):
            plt.subplot('1'+str(n)+str(num+1))
            l,k=np.shape(therank[:,:,num])
            comeback=[]
            for j in range(k):
                if order.index(j)+1 in rank:
                    comeback.append(j)
                else:
                    plt.plot(range(20,(ts+1)*20,20),therank[:ts,j,num],color = '0.75')
            for j in comeback:     
                plt.plot(range(20,(ts+1)*20,20),therank[:ts,j,num])
            plt.ylim(121,0)
            plt.xlim([20,1020])
            plt.title(label[num])
            if num==0:
                plt.ylabel('rank')
            plt.xlabel('respondent')
        plt.draw()
    fig.set_size_inches(size[0], size[1])
    anim = animation.FuncAnimation(fig, updatefig, 52)
    anim.save(name+'.mp4', fps=5)
    plt.clf()
    plt.close('all')

In [22]:
rankanim(indices=[0,5],rank=np.arange(1,11),iterval=9,items='120',mis=0,name='rankingcomp2')
#rankanim(indices=[0,5,3],rank=np.arange(1,11),iterval=9,items='120',mis=0,name='rankingcomp3')
#rankanim(indices=[0,5,3],rank=np.arange(1,11),iterval=9,items='120',mis=1,name='rankingcomp3mis')

In [67]:
def betaplot(cols=[0],rank=[1],items='120',mis=0,name='name.pdf'):
    colors=['b','g','r','c']
    where,listoffiles=getfiles(start="bhist",items=items,mis=mis)
    whichcol=whichcols(rank)
    newcol='Item'+str(int(whichcol[0][1:])-2)
    truth=pd.read_csv('../MABMaxDiff/HB_120items.csv')[newcol].get_values().mean()
    z=0
    for col in cols:
        f=listoffiles[col]
        data=pd.read_csv(where+'/'+f)
        betas=data[data['V2']==1020][whichcol].get_values()[:,0]
        label=createlabel(f)
        x=np.linspace(truth-3,truth+3,200)
        dens=KernelDensity(bandwidth=.5)
        dens.fit(betas[:, np.newaxis])
        points=dens.score_samples(x[:, np.newaxis])
        plt.fill_between(x, 0, np.exp(points),facecolor=colors[z], alpha=0.5,label=label)
        z+=1
    plt.axvline(truth,color='k')
    #plt.ylabel('Value')
    plt.legend(loc='best',fontsize='small')
    plt.xlabel('Utility')
    plt.xlim(-4,4)
    plt.ylim(0,1)
    #plt.title('Beta Distribution of rank '+str(rank[0])+' item' )
    plt.savefig(name, dpi=100)
    plt.close()

In [68]:
betaplot(cols=[0,5,8],rank=[1],items='120',mis=0,name='plots/endbetadis1.pdf')
betaplot(cols=[0,5,8],rank=[30],items='120',mis=0,name='plots/endbetadis30.pdf')
betaplot(cols=[0,5,8],rank=[60],items='120',mis=0,name='plots/endbetadis60.pdf')
betaplot(cols=[0,5,8],rank=[90],items='120',mis=0,name='plots/endbetadis90.pdf')

In [33]:
def stopplot(cols=[0,5],value=5,num=3,items='120',mis=0,stop=True,name='name.pdf'):
    colors=['b','g','r','c']
    where,listoffiles=getfiles(start="stop",items=items,mis=mis)
    for i,col in enumerate(cols):
        f=listoffiles[col]
        data=pd.read_csv(where+'/'+f)
        if stop:
            values=data['stop'+str(value)+'rem'+str(num)].get_values()
        else:
            values=data['util'+str(value)+'rem'+str(num)].get_values()
        
        label=createlabel(f)
        plt.hist(values,alpha=.5,color=colors[i],label=label)
        
    #plt.ylabel('Value')
    #plt.xlabel('respondent')
    if stop:
        #plt.title('When survey stops for '+str(num)+ ' items using '+str(value/100)+ ' rule')
        plt.xlabel('Respondent')
    else:
        #plt.title('True utility when survey stops for '+str(num)+ ' items using '+str(value/100)+ ' rule')
        plt.xlabel('Percent of True Utility')
    plt.legend(loc='best',fontsize='small')
    plt.savefig(name, dpi=100)
    plt.close()

In [34]:
stopplot([0,5,8],value=5,num=3,stop=True,name='plots/stop120items53.pdf')
stopplot([0,5,8],value=2,num=3,stop=True,name='plots/stop120items23.pdf')
stopplot([0,5,8],value=5,num=10,stop=True,name='plots/stop120items510.pdf')
stopplot([0,5,8],value=2,num=10,stop=True,name='plots/stop120items210.pdf')
stopplot([0,5,8],value=5,num=3,stop=False,name='plots/util120items53.pdf')
stopplot([0,5,8],value=2,num=3,stop=False,name='plots/util120items23.pdf')
stopplot([0,5,8],value=5,num=10,stop=False,name='plots/util120items510.pdf')
stopplot([0,5,8],value=2,num=10,stop=False,name='plots/util120items210.pdf')

In [54]:
stopplot([0,1,2],items='300',value=5,num=3,stop=True,name='plots/stop300items53.pdf')
stopplot([0,1,2],items='300',value=2,num=3,stop=True,name='plots/stop300items23.pdf')
stopplot([0,1,2],items='300',value=5,num=10,stop=True,name='plots/stop300items510.pdf')
stopplot([0,1,2],items='300',value=2,num=10,stop=True,name='plots/stop300items210.pdf')
stopplot([0,1,2],items='300',value=5,num=3,stop=False,name='plots/util300items53.pdf')
stopplot([0,1,2],items='300',value=2,num=3,stop=False,name='plots/util300items23.pdf')
stopplot([0,1,2],items='300',value=5,num=10,stop=False,name='plots/util300items510.pdf')
stopplot([0,1,2],items='300',value=2,num=10,stop=False,name='plots/util300items210.pdf')

In [49]:
where,listoffiles=getfiles(start="stop",items='300')

In [50]:
listoffiles

['stop (fixed_sparse - 300 items - 30 items per resp - misinf 0 ).csv',
 'stop (TS - 300 items - 30 items per resp - split 20 TS 10 TSx10 - misinf 0 ).csv',
 'stop (TS - 300 items - 30 items per resp - split 30 TS 0 TSx1 - misinf 0 ).csv']

In [40]:
def stopplotcor(cols=[0,5],value=5,num=3,items='120',mis=0,bins=10,name='name.pdf'):
    colors=['r','b','g','c']
    where,listoffiles=getfiles(start="stop",items=items,mis=mis)
    for i,col in enumerate(cols):
        f=listoffiles[col]
        data=pd.read_csv(where+'/'+f)
        stop=data['stop'+str(value)+'rem'+str(num)].get_values()
        util=data['util'+str(value)+'rem'+str(num)].get_values()
        
        label=createlabel(f)
        plt.plot(stop,util,color=colors[i],label=label,marker='.',linestyle='None')
        
    plt.ylabel('util')
    plt.xlabel('stop')
    plt.title('connection between util and stop for '+str(num)+ ' items using '+str(value/100)+ ' rule')
    plt.legend(loc='best',fontsize='small')
    plt.savefig(name, dpi=100)
    plt.close()

In [55]:
stopplotcor(cols=[0,1,2],value=5,num=10,items='300')

In [36]:
def stopmean(cols=[0,5],value=5,num=3,items='120',mis=0):
    colors=['r','b','g','c']
    where,listoffiles=getfiles(start="stop",items=items,mis=mis)
    for i,col in enumerate(cols):
        f=listoffiles[col]
        data=pd.read_csv(where+'/'+f)
        stop=data['stop'+str(value)+'rem'+str(num)].get_values()
        util=data['util'+str(value)+'rem'+str(num)].get_values()
        label=createlabel(f)
        print(label)
        print('mean stop:',np.mean(stop))
        print('mean util:',np.mean(util))

In [43]:
stopmean(cols=[0,5,8],value=2,mis=0,num=3,items='120')

Fixed Express
mean stop: 980.8
mean util: 0.975844446303
TS $\epsilon$=0.333 $\delta$=10
mean stop: 440.6
mean util: 0.99672412237
TS $\epsilon$=0.0 $\delta$=1
mean stop: 606.0
mean util: 0.996278651102


In [46]:
stopmean(cols=np.arange(0,3),value=2,mis=0,num=3,items='300')

Fixed Express
mean stop: 1020.0
mean util: 0.903718456084
TS $\epsilon$=0.333 $\delta$=10
mean stop: 674.2
mean util: 0.992719902665
TS $\epsilon$=0.0 $\delta$=1
mean stop: 929.4
mean util: 0.988091390873
